<a href="https://colab.research.google.com/github/deekshak77/OCR_TASK/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber pandas pytesseract pdf2image openpyxl
!apt-get install -y poppler-utils



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (313 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124950 files and directories currently installed.)
Preparing to 

In [2]:
import pdfplumber
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from google.colab import files

def extract_tables_from_pdf(pdf_path, page_number):
    tables_data = []
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number - 1]
        tables = page.extract_tables()
        for table in tables:
            df = pd.DataFrame(table)
            tables_data.append(df)
    return tables_data

def perform_ocr_on_page(pdf_path, page_number):
    images = convert_from_path(pdf_path, first_page=page_number, last_page=page_number)
    extracted_text = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_text.append(text)
    return "\n".join(extracted_text)

pdf_files = {
    "cardio_structured.pdf": 6,
    "prot_sap_102.pdf": 50,
    "prot_sap_1.pdf": 14
}

output_file = "/content/extracted_tables.xlsx"
writer = pd.ExcelWriter(output_file, engine='openpyxl')

uploaded = files.upload()

for pdf_name, page_number in pdf_files.items():
    print(f"Processing {pdf_name} - Page {page_number}...")
    pdf_path = f"/content/{pdf_name}"

    if pdf_name in uploaded:
        try:
            tables = extract_tables_from_pdf(pdf_path, page_number)

            if tables:
                for i, table in enumerate(tables):
                    table.to_excel(writer, sheet_name=f"{pdf_name}_Page{page_number}_Table{i+1}", index=False)
            else:
                extracted_text = perform_ocr_on_page(pdf_path, page_number)
                df_text = pd.DataFrame([extracted_text.split("\n")])
                df_text.to_excel(writer, sheet_name=f"{pdf_name}_Page{page_number}_OCR", index=False)

        except Exception as e:
            print(f"Error processing {pdf_name} - Page {page_number}: {e}")
    else:
        print(f"Error: {pdf_name} not found. Please upload the file.")

writer.close()
print(f"Extraction completed! Saved as {output_file}")
files.download(output_file)


Saving cardio_structured.pdf to cardio_structured.pdf
Saving prot_sap_1.pdf to prot_sap_1.pdf
Saving prot_sap_102.pdf to prot_sap_102.pdf
Saving sample.xlsx to sample.xlsx
Processing cardio_structured.pdf - Page 6...


/usr/local/lib/python3.11/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing prot_sap_102.pdf - Page 50...
Processing prot_sap_1.pdf - Page 14...
Extraction completed! Saved as /content/extracted_tables.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>